In [0]:
%run "/Src/Include/utility"

Running data set firt time

In [0]:
from pyspark. sql. functions import *
from pyspark.sql.window import Window
win = Window.orderBy('CustomerID')
df = spark.read.format("csv").options(header=True,inferSchema='True').load('/FileStore/tables/Input/Customer.csv') \
    .withColumn('Effective_start_date',date_format(current_date(), "yyyy-MM-dd")) \
    .withColumn('Effective_end_date',lit('2999-01-01')) \
    .withColumn('Current_flag',lit(True))

df.write.mode('overwrite').options(header=True).format('parquet').save('/FileStore/tables/Output/Customer.parquet')

In [0]:
df_currect = spark.read.format("csv").options(header=True,inferSchema='True').load('/FileStore/tables/Customer-2.csv')

df_history = spark.read.format("parquet").options(header=True,inferSchema='True').load('/FileStore/tables/Output/Customer.parquet')

Open_records = df_history.where(col("Current_flag")==lit(True))
type2_cols = ["CompanyName", "EmailAddress", "Phone", "ZipCode"]
scd2_cols = ["effective_date","expiration_date","current_flag"]

In [0]:
df_history_open_hash= column_suffix(get_hash(Open_records, type2_cols),suffix="_history", append=True)
df_current_hash = column_suffix(get_hash(df_currect, type2_cols),suffix="_Current", append=True)

['CustomerID_history', 'Title_history', 'FirstName_history', 'LastName_history', 'CompanyName_history', 'EmailAddress_history', 'Phone_history', 'ZipCode_history', 'Effective_start_date_history', 'Effective_end_date_history', 'Current_flag_history', 'hash_md5_history']
['CustomerID_Current', 'Title_Current', 'FirstName_Current', 'LastName_Current', 'CompanyName_Current', 'EmailAddress_Current', 'Phone_Current', 'ZipCode_Current', 'hash_md5_Current']


In [0]:
df_result = df_history_open_hash.join(df_current_hash, col("CustomerID_current") ==  col("CustomerID_history"), how="full_outer") \
    .withColumn("Action",when(col("hash_md5_history") ==  col("hash_md5_current"),"Nochange") \
        .when(col("CustomerID_history").isNull(),"Insert") \
            .when(col("CustomerID_Current").isNull(),'Delete') \
                .otherwise("update") )
    
'''\
                    .withColumn('Effective_start_date',date_format(current_date(), "yyyy-MM-dd")) \
    .withColumn('Effective_end_date',lit('2999-01-01')) \
    .withColumn('Current_flag',lit(True)) '''   

#df_result.filter(col("CustomerID_Current").isNull())

Out[30]: '                    .withColumn(\'Effective_start_date\',date_format(current_date(), "yyyy-MM-dd"))     .withColumn(\'Effective_end_date\',lit(\'2999-01-01\'))     .withColumn(\'Current_flag\',lit(True)) '

In [0]:
Df_New_record = column_suffix(df_result.filter(col("Action")=="Insert"),suffix="_Current", append=False).select(df_currect.columns) \
    .withColumn('Effective_start_date',date_format(current_date(), "yyyy-MM-dd")) \
    .withColumn('Effective_end_date',lit('2999-01-01')) \
    .withColumn('Current_flag',lit(True))

Df_New_record.show()


+----------+-----+---------+--------+-----------+--------------------+-------------------+-------+--------------------+------------------+------------+
|CustomerID|Title|FirstName|LastName|CompanyName|        EmailAddress|              Phone|ZipCode|Effective_start_date|Effective_end_date|Current_flag|
+----------+-----+---------+--------+-----------+--------------------+-------------------+-------+--------------------+------------------+------------+
|        14|  Mr.|   Pushpa|  Tiwari|    Walmart| rahuljain@gmail.com|       346-235-0222| 470139|          2023-08-26|        2999-01-01|        true|
+----------+-----+---------+--------+-----------+--------------------+-------------------+-------+--------------------+------------------+------------+



In [0]:
Df_old_record = column_suffix(df_result.filter(col("Action")=="Nochange"),suffix="_Current", append=False).select(df_currect.columns) \
    .withColumn('Effective_start_date',date_format(current_date(), "yyyy-MM-dd")) \
    .withColumn('Effective_end_date',lit('2999-01-01')) \
    .withColumn('Current_flag',lit(True))

Df_old_record.show()

+----------+-----+---------+--------+-----------+--------------------+-------------------+-------+--------------------+------------------+------------+
|CustomerID|Title|FirstName|LastName|CompanyName|        EmailAddress|              Phone|ZipCode|Effective_start_date|Effective_end_date|Current_flag|
+----------+-----+---------+--------+-----------+--------------------+-------------------+-------+--------------------+------------------+------------+
|         1|  Mr.|  Pritesh|   Joshi| CloudCover|PriteshJoshi@gmai...|       150-555-0127| 431122|          2023-08-26|        2999-01-01|        true|
|         2|  Mr.|    Omkar|   Joshi|     Google|OmkarJoshigoofle@...|       926-555-0159| 411046|          2023-08-26|        2999-01-01|        true|
|         3|  Mr.|  Shubham|    Mule|     Fassos|ShubhamMule@gmail...|       112-555-0191| 431123|          2023-08-26|        2999-01-01|        true|
|         4|  Mr.|  Kshitij|  Jadhav|      Wipro|KshitijJadhav@gma...|1 (11) 500 555-013

In [0]:
Df_deleted_record = column_suffix(df_result.filter(col("Action")=="Delete"),suffix="_history", append=False).select(df_currect.columns) \
    .withColumn('Effective_start_date',date_format(current_date(), "yyyy-MM-dd")) \
    .withColumn('Effective_end_date',lit('2999-01-01')) \
    .withColumn('Current_flag',lit(False))
Df_deleted_record.show()

+----------+-----+---------+--------+-----------+--------------------+-------------------+-------+--------------------+------------------+------------+
|CustomerID|Title|FirstName|LastName|CompanyName|        EmailAddress|              Phone|ZipCode|Effective_start_date|Effective_end_date|Current_flag|
+----------+-----+---------+--------+-----------+--------------------+-------------------+-------+--------------------+------------------+------------+
|        13|  Mr.|   Pushpa|  Tiwari|    Walmart| rahuljain@gmail.com|       346-235-0222| 470139|          2023-08-26|        2999-01-01|       false|
+----------+-----+---------+--------+-----------+--------------------+-------------------+-------+--------------------+------------------+------------+



In [0]:
Df_update_record = column_suffix(df_result.filter(col("Action")=="update"),suffix="_history", append=False).select(df_currect.columns) \
    .withColumn('Effective_start_date',date_format(current_date(), "yyyy-MM-dd")) \
    .withColumn('Effective_end_date',date_format(current_date(), "yyyy-MM-dd")) \
    .withColumn('Current_flag',lit(False)) \
    .union (column_suffix(df_result.filter(col("Action")=="update"),suffix="_Current", append=False).select(df_currect.columns) \
    .withColumn('Effective_start_date',date_format(current_date(), "yyyy-MM-dd")) \
    .withColumn('Effective_end_date',lit('2999-01-01')) \
    .withColumn('Current_flag',lit(True)))

Df_update_record.show()

+----------+-----+---------+--------+-----------+-----------------+-------------------+-------+--------------------+------------------+------------+
|CustomerID|Title|FirstName|LastName|CompanyName|     EmailAddress|              Phone|ZipCode|Effective_start_date|Effective_end_date|Current_flag|
+----------+-----+---------+--------+-----------+-----------------+-------------------+-------+--------------------+------------------+------------+
|        12|  Mr.|  Sandeep|  Tiwari|    Walmart|Sandeep@gmail.com|       346-235-0222| 470139|          2023-08-26|        2023-08-26|       false|
|        12|  Mr.|  Sandeep|Tripathi|        TCS|Sandeep@gmail.com|       346-235-0222| 470139|          2023-08-26|        2999-01-01|        true|
+----------+-----+---------+--------+-----------+-----------------+-------------------+-------+--------------------+------------------+------------+



In [0]:
final_df = Df_New_record.union(Df_deleted_record).union(Df_update_record).union(Df_old_record)

final_df.write.mode('overwrite').options(header=True).format('parquet').save('/FileStore/tables/Output1/Customer1.parquet')

df= spark.read.format("parquet").options(header=True,inferSchema='True').load('/FileStore/tables/Output/Customer1.parquet')
